In [1]:
import pandas as pd
import os
import datetime
import holidays
import csv
import io
from dataclasses import dataclass, asdict

In [344]:
@dataclass
class PowerData:
    power_plant: str = ""
    total_generation: float = 0 # 總產電
    total_transfered: float = 0  # 總轉供
    residual_power: int = 0 # 總餘電
    time_slot: str = 0

@dataclass
class ClientData:
    client_id: str
    total_utility_power: float = 0  # 總市電
    total_quota: float = 0  # 總綠電轉供 (stage1 + stage2)
    time_slot: str = 0

@dataclass
class Plant:
    plant_name: str
    total_transfered: float = 0  # 總轉供
    residual_power: int = 0
    time_slot: str = 0


class PlantRecorder:
    def __init__(self, plant_names):
        self.hd_plant_dict = {name: Plant(name) for name in plant_names}

    def add_power_transfered(self, plant_name, power_transfered, time_slot, residual_power):
        obj = self.hd_plant_dict[plant_name]
        obj.total_transfered = float(power_transfered)
        obj.time_slot = time_slot
        obj.residual_power = residual_power

    def add_power_generation(self, plant_name):
        obj = self.hd_plant_dict[plant_name]

    def get_power_transfered(self, plant_name):
        return self.hd_plant_dict[plant_name].total_transfered

    def get_residual_power(self, plant_name):
        return self.hd_plant_dict[plant_name].residual_power

    def get_plant_name(self, plant_name):
        return self.hd_plant_dict[plant_name].plant_name

    def summarize(self):
        for name, obj in self.hd_plant_dict.items():
            print(asdict(obj))

In [345]:
def __load_power_plant_dicts(filepath):
    power_plant_dicts = {}
    with open(filepath + os.path.sep + 'plant_information.csv', 'r', encoding="utf-8-sig") as f:
        rows = csv.reader(f, delimiter=',')
        _headers = next(rows)
        for row in rows:
            power_plant_dicts[row[0]] = {}
            # HDRE asks to round FIT to 4 digits
            power_plant_dicts[row[0]]["surplus_price"] = round(float(row[1]), 4)

            # HDRE asks to round capacity to 2 digits
            power_plant_dicts[row[0]]["capacity"] = round(float(row[2]), 2)
            power_plant_dicts[row[0]]["power_plant_id"] = row[3]
            power_plant_dicts[row[0]]["surplus_power"] = 0
    return power_plant_dicts

def __load_power_client_dicts(filepath):
    power_plant_dicts = {}
    with open(filepath + os.path.sep + 'client_information.csv', 'r', encoding="utf-8-sig") as f:
        rows = csv.reader(f, delimiter=',')
        _headers = next(rows)
        for row in rows:
            power_plant_dicts[row[1]] = {}
            power_plant_dicts[row[1]]["綠電售價"] = round(float(row[6]), 4)
    return power_plant_dicts

def read_predefined_contracts(csv_fn):
    assert csv_fn.endswith(".csv"), f"Wrong csv_fn: {csv_fn}"
    with io.open(csv_fn, encoding="utf-8-sig") as _fp:
        reader = csv.reader(_fp, delimiter=",")
        rows = [row for row in reader]
    plant_name = rows[0][1]
    plant_generation = rows[0][2]
    docs = []
    for row in rows[1:]:
        doc = {
            'type': row[0],
            plant_name: row[1],
            plant_generation: row[2]
        }
        docs.append(doc)

    return docs

In [346]:
power_plant_dict = __load_power_plant_dicts('./contract_3_5_month_minutes/')
print(power_plant_dict)
power_client_dict = __load_power_client_dicts('./contract_3_5_month_minutes/')
power_client_dict

{'電廠001': {'surplus_price': 5.0004, 'capacity': 9000.0, 'power_plant_id': '20655320001', 'surplus_power': 0}, '電廠002': {'surplus_price': 4.9552, 'capacity': 12000.0, 'power_plant_id': '20655320002', 'surplus_power': 0}}


{'64-67-0001-13-1': {'綠電售價': 5.5376},
 '64-67-0002-13-1': {'綠電售價': 6.2255},
 '64-67-0003-13-1': {'綠電售價': 5.9575},
 '64-67-0004-13-1': {'綠電售價': 6.0537},
 '64-67-0005-13-1': {'綠電售價': 5.5821}}

In [347]:
plant_record = PlantRecorder(power_plant_dict.keys())
plant_record.summarize()

{'plant_name': '電廠001', 'total_transfered': 0, 'residual_power': 0, 'time_slot': 0}
{'plant_name': '電廠002', 'total_transfered': 0, 'residual_power': 0, 'time_slot': 0}


In [353]:
def analyze_power(power_generation, power_consumption):
    type_list = []
    type_list1 = []
    type_list2 = []

    type_list_c = []
    type_list1_c = []
    type_list2_c = []

    df_g = read_predefined_contracts(power_generation)
    df_c = read_predefined_contracts(power_consumption)

    # type distribution
    for i in df_g:
        if i['type'] == '0':
            type_list.append(i)
        elif i['type'] == '1':
            type_list1.append(i)
        else:
            type_list2.append(i)
    for j in df_c:
        if j['type'] == '0':
            type_list_c.append(j)
        elif i['type'] == '1':
            type_list1_c.append(i)
        else:
            type_list2_c.append(i)

    return type_list, type_list1, type_list2, type_list_c, type_list1_c, type_list2_c


def get_power_max(p_list):
    return sorted(p_list, key=lambda x: x['power_generation'], reverse=True)

def get_consumption_max(plant_dict):
    sorting_list = sorted({d['綠電售價']: (k, d) for k, d in plant_dict.items()}, reverse=True)
    plant_dict = {d['綠電售價']: (k, d) for k, d in plant_dict.items()}
    plant_dict = dict(map(plant_dict.get, sorting_list))
    return plant_dict
    # return sorted(c_list, key=lambda x: x['綠電售價'], reverse=True)

def cal_distribution(power_dict, consumption_dict, power_price):
    for i in power_dict:
        doc =[
            PowerData(
                power_plant=i.get('power_plant'),
                total_generation=i.get('power_generation')
            )
        ]
        print(doc)
    # print(power_dict, consumption_dict, power_price)

In [354]:
p_g_0, _, p_g_2, p_c_0, _, p_c_2 = analyze_power('power_generation.csv', 'power_consumption.csv')
print(p_g_0)
print(p_c_0)

[{'type': '0', 'power_plant': '電廠001', 'power_generation': '447200'}, {'type': '0', 'power_plant': '電廠002', 'power_generation': '559000'}]
[{'type': '0', 'client_id': '64-67-0001-13-1', 'power_consumption': '379638'}, {'type': '0', 'client_id': '64-67-0002-13-1', 'power_consumption': '455565.6'}, {'type': '0', 'client_id': '64-67-0003-13-1', 'power_consumption': '303710.4'}, {'type': '0', 'client_id': '64-67-0004-13-1', 'power_consumption': '560350'}, {'type': '0', 'client_id': '64-67-0005-13-1', 'power_consumption': '2186722.5'}]


In [355]:
power_dict_list = get_power_max(p_g_0)
power_price_list = get_consumption_max(power_client_dict)

In [356]:
cal_distribution(power_dict_list, p_c_0, power_price_list)

[PowerData(power_plant='電廠002', total_generation='559000', total_transfered=0, residual_power=0, time_slot=0)]
[PowerData(power_plant='電廠001', total_generation='447200', total_transfered=0, residual_power=0, time_slot=0)]
